In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

file_path = 'data/Supplemental_Data_Raw_Genecounts.csv'

# Load the CSV file into a DataFrame.
df = pd.read_csv(file_path)
df = df.drop('Unnamed: 17', axis=1)
df = df.rename(columns={"Unnamed: 0": "mnra",})
df.set_index('mnra', inplace=True)
# Convert every non-index column to int
df = df.apply(pd.to_numeric, errors='coerce', downcast='integer')

# Pruning rows where all the values are 0.
# Assuming that you have columns 'x' and 'y', replace them with the actual names of your columns.
print(df.shape)
df = df[(df.loc[:, df.columns != 'index'] != 0).any(axis=1)]

In [ ]:
columns_to_check = df.columns.difference(['Unnamed: 0'])

# Pruning rows where all the values in columns_to_check are 0.
df = df[(df[columns_to_check] != 0).any(axis=1)]


In [ ]:
# Step 1: Calculate pairwise correlation
correlation_matrix = df.corr(method='pearson')



In [ ]:
correlation_matrix

In [ ]:
filtered_corr_matrix = correlation_matrix.drop(index='B2.3.4', columns='B2.3.4')

In [ ]:
sns.heatmap(filtered_corr_matrix, annot=False, cmap='viridis')
plt.title('sample-sample Correlation Heatmap')
plt.show()

In [ ]:
plt.figure()
sns.heatmap(correlation_matrix, annot=False, cmap='viridis')
plt.title('sample-sample Correlation Heatmap')
plt.show()

In [ ]:
# Calculate gene-gene correlation matrix in a single line
gene_correlation_matrix = df.T.corr(method='pearson')

In [ ]:
gene_correlation_matrix

In [ ]:
# Select first 100 genes for testing
subset_corr_matrix = gene_correlation_matrix.iloc[:400, :400]
threshold = 0.90  # bootstrapping ?
filtered_subset_corr_matrix = subset_corr_matrix[(abs(subset_corr_matrix) >= threshold) & (subset_corr_matrix != 1.0)]

G_subset = nx.Graph()

for gene1 in filtered_subset_corr_matrix.index:
    for gene2 in filtered_subset_corr_matrix.columns:
        correlation = filtered_subset_corr_matrix.loc[gene1, gene2]
        if not np.isnan(correlation):
            G_subset.add_edge(gene1, gene2, weight=correlation)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(subset_corr_matrix, annot=False, cmap='viridis')
plt.title('gene-gene Correlation Heatmap')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(filtered_subset_corr_matrix, annot=False, cmap='viridis')
plt.title('gene-gene Correlation Heatmap')
plt.show()

In [ ]:

pos_subset = nx.spring_layout(G_subset)
fig, ax = plt.subplots(figsize=(8, 8))
nx.draw(G_subset, pos_subset, ax=ax, with_labels=True, node_color="skyblue", node_size=1, font_size=10)
plt.title("Initial Co-expression Network (Subset)")
plt.show()

In [ ]:
import community  # Python Louvain method library

# First compute the best partition
partition = community.best_partition(G_subset)

# Create a new graph to represent the hierarchical structure
G_hierarchy = nx.Graph()

for node, mod_class in partition.items():
    G_hierarchy.add_edge(node, f"Module_{mod_class}")

# Generate layout and draw the hierarchical network
pos_hierarchy = nx.spring_layout(G_hierarchy)
fig, ax = plt.subplots(figsize=(8, 8))
nx.draw(G_hierarchy, pos_hierarchy, with_labels=True, node_color="skyblue", node_size=1000, font_size=10)
plt.title("Hierarchical Co-expression Network")
plt.show()


In [ ]:
# Initialize a list to hold the colors for each node
colors = [partition[node] for node in G_subset.nodes()]

# Create the layout and plot the network
pos_subset = nx.spring_layout(G_subset)
fig, ax = plt.subplots(figsize=(32, 32))
nx.draw(G_subset, pos_subset, ax=ax, with_labels=True, node_color=colors, cmap=plt.cm.rainbow, node_size=100, font_size=10)
plt.title("Co-expression Network with Community Structure (Subset)")
plt.show()


In [ ]:
# Initialize a list to hold the colors for each node
colors = [partition[node] for node in G_subset.nodes()]

# Create the layout and plot the network
pos_subset = nx.spring_layout(G_subset)
fig, ax = plt.subplots(figsize=(16, 16))
nx.draw(G_subset, pos_subset, ax=ax, with_labels=True, node_color=colors, cmap=plt.cm.rainbow, node_size=30, font_size=10, edge_color="grey", alpha=0.5)
plt.title("Co-expression Network with Community Structure (Subset)")
plt.show()

In [ ]:
# Select first 100 genes for testing
subset_corr_matrix = gene_correlation_matrix.iloc[:400, :400]
threshold = 0.90  # bootstrapping ?
filtered_subset_corr_matrix = subset_corr_matrix[(abs(subset_corr_matrix) >= threshold) & (subset_corr_matrix != 1.0)]

In [ ]:
filtered_subset_corr_matrix

In [ ]:
from graph_tool.all import *
import numpy as np

# Convert DataFrame to NumPy array
filtered_subset_corr_matrix_np = filtered_subset_corr_matrix.to_numpy()

# Initialize Graph object
g = Graph(directed=False)

# Add vertices
num_vertices = filtered_subset_corr_matrix_np.shape[0]
for _ in range(num_vertices):
    g.add_vertex()

# Initialize edge weights
e_weight = g.new_edge_property("double")

# Loop through the matrix to add edges and weights
for i in range(num_vertices):
    for j in range(i+1, num_vertices):
        if filtered_subset_corr_matrix_np[i, j] != 0:
            e = g.add_edge(i, j)
            e_weight[e] = filtered_subset_corr_matrix_np[i, j]

# Assign the edge weights to the graph
g.ep["weight"] = e_weight


In [ ]:
from graph_tool.all import *
import numpy as np

# Convert DataFrame to NumPy array
filtered_subset_corr_matrix_np = filtered_subset_corr_matrix.fillna(0).to_numpy()

# Initialize Graph object
g = Graph(directed=False)

# Add vertices
num_vertices = filtered_subset_corr_matrix_np.shape[0]
for _ in range(num_vertices):
    g.add_vertex()

# Initialize edge weights
e_weight = g.new_edge_property("double")

# Loop through the matrix to add edges and weights
for i in range(num_vertices):
    for j in range(i+1, num_vertices):
        if filtered_subset_corr_matrix_np[i, j] != 0:
            e = g.add_edge(i, j)
            e_weight[e] = filtered_subset_corr_matrix_np[i, j]

# Assign the edge weights to the graph
g.ep["weight"] = e_weight


In [ ]:
from graph_tool.draw import graph_draw

# 3D Visualization
graph_draw(g, 
           edge_color=[1,1,1,1],
           vertex_fill_color=[1,0,0,1],
           output_size=(800, 800),
           output="3D_network.png",
           bg_color=[0,0,0,1])
